In [ ]:
import numpy as np
import astropy.units as u

from astropy.table import Table, QTable, Column
from os import listdir, rename
from matplotlib import pyplot as plt


def perform_Gaia_query(G_lim):
    """Perform a query from Gaia DR2.

    The query will include all the stars brighter with G<G_lim that also have a 5-parameter
    astrometric solution together with BP and RP measurements. Their astrometric parameters,
    uncertainties and correlations together with G, BP and RP magnitudes are saved to a VOTable.
    The name of the VOTable file is returned.
    """
    from astroquery.gaia import Gaia

    astrometry = ('ra', 'dec', 'parallax', 'pmra', 'pmdec')
    columns_to_query = ''
    # Add astrometric measurements, uncertanties and correlations to the columns to be queried
    for i, col in enumerate(astrometry):
        columns_to_query += col+', '
        columns_to_query += col+'_error, '
        for col2 in astrometry[i+1:]:
            columns_to_query += col+'_'+col2+'_corr, '
    # Make sure the magnitudes in the three bands are queried too
    for band in ('g', 'bp', 'rp'):
        columns_to_query += 'phot_'+band+'_mean_mag, '
    # Remove the last comma
    columns_to_query = columns_to_query[:-2]
    job = Gaia.launch_job_async(f'SELECT {columns_to_query}\
                                 FROM gaiadr2.gaia_source\
                                 WHERE phot_g_mean_mag < {G_lim}\
                                     AND parallax IS NOT NULL\
                                     AND bp_rp IS NOT NULL\
                                 ;', dump_to_file=True)
    return job.outputFile


G_lim = 11
filename = f'brighter_than_{G_lim}.vot'
if filename not in listdir():
    print('Performing a query from Gaia archive. This might take a while...')
    rename(perform_Gaia_query(G_lim), filename)
else:
    print('Reading data from a stored file.')
data = QTable.read(filename)
# Astropy QTable has problems parsing the proper motion units, so we have to help it along.
for col in data.columns:
    if 'pm' in col and 'corr' not in col:
        data[col] = np.array(data[col])*u.Unit(str(data[col].unit))
print('Done.')

In [ ]:
def construct_covariance_matrix(stars):
    """Create the covariance matrix of the astrometric parameters of input stars."""
    covariance = np.ones((len(stars), 5, 5))
    astrometry = ('ra', 'dec', 'parallax', 'pmra', 'pmdec')
    for i, col in enumerate(astrometry):
        # This covariance matrix is set up as a numpy array, so we need to strip the units.
        covariance[:, i, :] *= np.array(stars[col+'_error'])[:, np.newaxis]
        covariance[:, :, i] *= np.array(stars[col+'_error'])[:, np.newaxis]
        for j in range(i+1, 5):
            covariance[:, i, j] *= np.array(stars[col+'_'+astrometry[j]+'_corr'])
            covariance[:, j, i] = covariance[:, i, j]
    return covariance


def convert_icrs_to_gal(stars):
    """Convert coordinates from ICRS to the Galactic system."""
    from coordTransform import transformIcrsToGal
    gal_params = ('l', 'b', 'pml', 'pmb')
    coords = np.array(stars['ra', 'dec', 'parallax', 'pmra', 'pmdec']).view((float, 5))
    covariance = construct_covariance_matrix(stars)
    gal_coords, gal_uncerts, gal_covariance = transformIcrsToGal(coords, 0, EqC=covariance)
    temp = QTable(gal_coords[:, np.array((0, 1, 3, 4))], names=gal_params)
    stars.add_columns(temp.columns.values())
    for i, name, unit, err_unit in zip((0, 1, 3, 4), gal_params,
                                       (u.deg, u.deg, u.mas/u.yr, u.mas/u.yr),
                                       (u.mas, u.mas, u.mas/u.yr, u.mas/u.yr)):
        stars[name] *= unit
        stars.add_column(Column(np.sqrt(gal_covariance[:, i, i])), name=name+'_error')
        # Right ascension and declination are in deg, but their uncertanties in mas.
        # We must ensure the same for galactic longitude and latitude.
        stars[name+'_error'] *= err_unit
    return stars


# Don't compute the galactic coordinates if they are already computed.
if 'b' not in data.columns:
    data = convert_icrs_to_gal(data)

In [ ]:
'''Calculating Tmean, taumean and vmeans'''

filters = [(data['parallax'] > 10*data['parallax_error']) &
                    (data['phot_bp_mean_mag'].value - data['phot_rp_mean_mag'].value > i) &
                    ((data['phot_bp_mean_mag'].value - data['phot_rp_mean_mag'].value) < i+0.1) for i in np.arange(-0.3,0.8,0.1)]
F = [data[filters[i]] for i in range(len(filters))]
N = [len(F[i]) for i in range(len(filters))]

app = [np.reshape(F[i]['phot_g_mean_mag'].value,(1,len(F[i]))) for i in range(len(N))]
p = [np.reshape(F[i]['parallax'].value,(1,len(F[i]))) for i in range(len(N))]
parallax = [np.reshape(F[i]['parallax'],(1,N[i])) for i in range(len(N))]
longitude = [F[i]['l'] for i in range(len(N))]
latitude = [F[i]['b'] for i in range(len(N))]
mu_l = [np.reshape(F[i]['pml'],(1,N[i])) for i in range(len(N))]
mu_b = [np.reshape(F[i]['pmb'],(1,N[i])) for i in range(len(N))]
r = [np.reshape(parallax[i].to(u.kpc, equivalencies=u.parallax()),(1,N[i])) for i in range(len(N))]
K = 4.7405 *u.km * u.yr / u.s / u.kpc / u.mas

l = [np.array([-np.sin(longitude[i]), np.cos(longitude[i]), np.zeros(len(F[i]))]) for i in range(len(N))]
b = [np.array([-np.sin(latitude[i])*np.cos(longitude[i]), -np.sin(latitude[i])*np.sin(longitude[i]), np.cos(latitude[i])]) for i in range(len(N))]


Tm = []
taum = []
v_m = []
for j in range(len(N)):
    t1 = (l[j]*K*mu_l[j])*r[j]
    t2 = (b[j]*K*mu_b[j])*r[j]
    tau = t1 + t2
    Tmean = np.zeros((3,3))
    for i in range(N[j]): #i = star
        u_1 = np.reshape(np.array([np.cos(latitude[j][i])*np.cos(longitude[j][i]), np.cos(latitude[j][i])*np.sin(longitude[j][i]), np.sin(latitude[j][i])]), (1,3))
        T = np.eye(3) - np.outer(u_1,np.transpose(u_1))
        Tmean += T

    Tmean /= N[j]
    Tm.append(Tmean)
    
    taumean = tau.mean(axis=1)
    taum.append(taumean)

    v_avv = np.matmul(np.linalg.inv(Tmean),taumean)
    v_m.append(v_avv)
    print(j)


In [ ]:
v_m

In [ ]:
'''Calculating B-matrix'''

Bm = []
for j in range(len(N)):
    B = np.zeros((3,3))
    for i in range(N[j]):    
        u_1 = np.reshape([np.cos(latitude[j][i])*np.cos(longitude[j][i]), np.cos(latitude[j][i])*np.sin(longitude[j][i]), np.sin(latitude[j][i])], (1,3))
        T = np.eye(3) - np.outer(u_1,np.transpose(u_1))

        l_1 = np.array([-np.sin(longitude[j][i]), np.cos(longitude[j][i]), np.zeros(len(F[j]))[i]])
        b_1 = np.array([-np.sin(latitude[j][i])*np.cos(longitude[j][i]), -np.sin(latitude[j][i])*np.sin(longitude[j][i]), np.cos(latitude[j][i])])

        t1 = (l_1*K*mu_l[j][0][i])*r[j][0][i]
        t2 = (b_1*K*mu_b[j][0][i])*r[j][0][i]
        tau = t1 + t2

        deltatau = tau - np.matmul(T,v_m[j])
        B = B + np.outer(deltatau,np.transpose(deltatau))

    B /= N[j]
    Bm.append(B)
    print(j)

In [ ]:
Bm

In [ ]:
'''Calculating D matrix, dispersion and non-redundant elements'''

Dm = []
disp = []
dm = []
for j in range(len(N)):  
    D = np.zeros((3,3))*u.km**2/(u.s**2)
    for i in range(3):
        for k in range(3):
            for m in range(3):
                for n in range(3):
                    if m <= n:
                        D[m,n] += Bm[j][i,k]/(Tm[j][k,m]*Tm[j][i,n])

    Dm.append(D)
    dispersions = np.diag(D)
    disp.append(dispersions)
    d = D[D!=0]
    dm.append(d)
disp

In [ ]:
Tm

In [ ]:
'''First try'''
Amean = []
displist = []
for i in range(len(N)):
    T_11 = Tm[i][0,0]
    T_12 = Tm[i][0,1]
    T_13 = Tm[i][0,2]
    T_21 = Tm[i][1,0]
    T_22 = Tm[i][1,1]
    T_23 = Tm[i][1,2]
    T_31 = Tm[i][2,0]
    T_32 = Tm[i][2,1]
    T_33 = Tm[i][2,2]
    A = np.array([[T_11*T_11, T_11*T_12 + T_12*T_11, T_11*T_13 + T_13*T_11, T_12*T_12, T_12*T_13 + T_13*T_12, T_13*T_13],
                 [T_11*T_21, T_11*T_22 + T_12*T_21, T_11*T_23 + T_13*T_21, T_12*T_22, T_12*T_23 + T_13*T_22, T_13*T_23],
                 [T_11*T_31, T_11*T_32 + T_12*T_31, T_11*T_33 + T_13*T_31, T_12*T_32, T_12*T_33 + T_13*T_32, T_13*T_33],
                 [T_21*T_21, T_21*T_22 + T_22*T_21, T_21*T_23 + T_23*T_21, T_22*T_22, T_22*T_23 + T_23*T_22, T_23*T_23],
                 [T_21*T_31, T_21*T_32 + T_22*T_31, T_21*T_33 + T_23*T_31, T_22*T_32, T_22*T_33 + T_23*T_32, T_23*T_33],
                 [T_31*T_31, T_31*T_32 + T_32*T_31, T_31*T_33 + T_33*T_31, T_32*T_32, T_32*T_33 + T_33*T_32, T_33*T_33]])
    B11 = Bm[i][0][0].value
    B12 = Bm[i][0][1].value
    B13 = Bm[i][0][2].value
    B22 = Bm[i][1,1].value
    B23 = Bm[i][1,2].value
    B33 = Bm[i][2,2].value
    
    b = np.array([[B11,B12,B13,B22,B23,B33]])
    Ainv = np.linalg.inv(A)
    Am = Ainv.mean(axis = 0)
    dispers = np.outer(Ainv,b)
    displist.append(dispers)
  
    
    
disp_x = []
disp_y = []
disp_z = []
Dlist = [] #Non-redunant elements

for i in range(len(N)):
    Dlist.append(displist[i][0])
    disp_x.append(displist[i][0][0])
    disp_y.append(displist[i][0][3])
    disp_z.append(displist[i][0][5])


In [ ]:
Amean = []
displist = []
for i in range(len(N)):
    T_11 = Tm[i][0,0]
    T_12 = Tm[i][0,1]
    T_13 = Tm[i][0,2]
    T_21 = Tm[i][1,0]
    T_22 = Tm[i][1,1]
    T_23 = Tm[i][1,2]
    T_31 = Tm[i][2,0]
    T_32 = Tm[i][2,1]
    T_33 = Tm[i][2,2]
    A = np.array([[T_11*T_11, T_11*T_12 + T_12*T_11, T_11*T_13 + T_13*T_11, T_12*T_12, T_12*T_13 + T_13*T_12, T_13*T_13],
                 [T_11*T_21, T_11*T_22 + T_12*T_21, T_11*T_23 + T_13*T_21, T_12*T_22, T_12*T_23 + T_13*T_22, T_13*T_23],
                 [T_11*T_31, T_11*T_32 + T_12*T_31, T_11*T_33 + T_13*T_31, T_12*T_32, T_12*T_33 + T_13*T_32, T_13*T_33],
                 [T_21*T_21, T_21*T_22 + T_22*T_21, T_21*T_23 + T_23*T_21, T_22*T_22, T_22*T_23 + T_23*T_22, T_23*T_23],
                 [T_21*T_31, T_21*T_32 + T_22*T_31, T_21*T_33 + T_23*T_31, T_22*T_32, T_22*T_33 + T_23*T_32, T_23*T_33],
                 [T_31*T_31, T_31*T_32 + T_32*T_31, T_31*T_33 + T_33*T_31, T_32*T_32, T_32*T_33 + T_33*T_32, T_33*T_33]])
    B11 = Bm[i][0,0].value
    B12 = Bm[i][0,1].value
    B13 = Bm[i][0,2].value
    B22 = Bm[i][1,1].value
    B23 = Bm[i][1,2].value
    B33 = Bm[i][2,2].value
    
    b = np.array([[B11,B12,B13,B22,B23,B33]])
    print(T_11)
    #bm = np.mean(b)
    #print(A)
    Ainv = np.linalg.inv(A)
    Am = Ainv.mean(axis = 0)
    dispers = np.outer(Ainv,b)
    displist.append(dispers)
  
    
    
disp_x = []
disp_y = []
disp_z = []
for i in range(len(N)):
    print(displist[i][0])
    disp_x.append(displist[i][0][0])
    disp_y.append(displist[i][0][3])
    disp_z.append(displist[i][0][5])
  
    




In [ ]:
Amean = np.zeros((11,6,6))
displist = []
for i in range(len(N)):
    T_11 = Tm[i][0,0]
    T_12 = Tm[i][0,1]
    T_13 = Tm[i][0,2]
    T_21 = Tm[i][1,0]
    T_22 = Tm[i][1,1]
    T_23 = Tm[i][1,2]
    T_31 = Tm[i][2,0]
    T_32 = Tm[i][2,1]
    T_33 = Tm[i][2,2]
    A = np.array([[T_11*T_11, T_11*T_12 + T_12*T_11, T_11*T_13 + T_13*T_11, T_12*T_12, T_12*T_13 + T_13*T_12, T_13*T_13],
                 [T_11*T_21, T_11*T_22 + T_12*T_21, T_11*T_23 + T_13*T_21, T_12*T_22, T_12*T_23 + T_13*T_22, T_13*T_23],
                 [T_11*T_31, T_11*T_32 + T_12*T_31, T_11*T_33 + T_13*T_31, T_12*T_32, T_12*T_33 + T_13*T_32, T_13*T_33],
                 [T_21*T_21, T_21*T_22 + T_22*T_21, T_21*T_23 + T_23*T_21, T_22*T_22, T_22*T_23 + T_23*T_22, T_23*T_23],
                 [T_21*T_31, T_21*T_32 + T_22*T_31, T_21*T_33 + T_23*T_31, T_22*T_32, T_22*T_33 + T_23*T_32, T_23*T_33],
                 [T_31*T_31, T_31*T_32 + T_32*T_31, T_31*T_33 + T_33*T_31, T_32*T_32, T_32*T_33 + T_33*T_32, T_33*T_33]])
    B11 = Bm[j][0][0].value
    B12 = Bm[j][0][1].value
    B13 = Bm[j][0][2].value
    B22 = Bm[j][1,1].value
    B23 = Bm[j][1,2].value
    B33 = Bm[j][2,2].value
    
    b = np.array([[B11,B12,B13,B22,B23,B33]])
    Am = A.mean(axis = 1)
    print(A.shape)
    print(Am.shape)
    #Ainv = np.linalg.inv(A)
    #Amean.append(Ainv)
    #Amean+= Ainv

    
#disp_x = []
#disp_y = []
#disp_z = []
#for i in range(len(N)):
#    disp_x.append(displist[i][0][0])
#    disp_y.append(displist[i][0][3])
#    disp_z.append(displist[i][0][5])
    

In [ ]:
'''Calculating maximum absolut magnitude and minimum parallax'''

M_Gl = []
index = []
p_min = []
G_min = []
for i in range(len(N)):
    M_G = app[i] + 5*np.log10(p[i]/100)
    M_Gl.append(M_G)

for i in range(len(N)):   
    index = (app[i] == np.amax(app[i]))
    p_min.append(parallax[i][0][index[0]])
    G_min.append(M_Gl[i][0][index[0]])

p_min,G_min

In [ ]:
'''Calculate vertex deviations'''
verticess = []
for j in range(len(N)):
    vert_d = 0.5*np.arctan((2*Dlist[j][1])/(Dlist[j][0] - Dlist[j][3]))
    verticess.append(vert_d)
verticess

In [ ]:
'''Calculate minimum distances in kpc??'''
distlist = []
for j in range(len(N)):
    dists = np.amin(r[j])
    distlist.append(dists)
distlist

In [ ]:
'''Calculate mean colours'''

bprp = [(F[i]['phot_bp_mean_mag']*u.mag - F[i]['phot_rp_mean_mag']*u.mag) for i in range(len(N))]
cm = []
for j in range(len(N)):
    cm.append(np.mean(bprp[j]))   
cm

In [ ]:
'''Plotting'''

colorinterval = [col for col in np.arange(-0.3,0.8,0.1)]
ages = [1.26e8, 2.51e8, 5.01e8, 6.31e8, 7.94e8, 1e9, 1.26e9, 1.58e9, 2.51e9, 3.98e9, 1e10]
meanages = []
for i in ages:
    meanages.append(i/2)
u_av = [v_m[i][0].value for i in range(len(N))]
v_av = [v_m[i][1].value for i in range(len(N))]
w_av = [v_m[i][2].value for i in range(len(N))]

disp_u = [disp[i][0].value for i in range(len(N))]
disp_v = [disp[i][1].value for i in range(len(N))]
disp_w = [disp[i][2].value for i in range(len(N))]

plt.figure(1)
plt.rcParams['font.size'] = 20
fig = plt.figure(figsize=(20, 20))
fig.add_subplot(221)
plt.title('Average velocity vs Colour')
plt.xlabel(r'bp-rp', fontsize = 18)
plt.ylabel('Avergae velocity [km/s]', fontsize = 18)
plt.plot(colorinterval,u_av, '--', label = r'$\langle u \rangle$')
plt.plot(colorinterval,v_av, '--', label = r'$\langle v \rangle$')
plt.plot(colorinterval,w_av, '--', label = r'$\langle w \rangle$')
plt.legend()

fig.add_subplot(222)
plt.title('Average velocity vs mean age')
plt.xscale('log')
plt.xlabel('Mean age', fontsize = 18)
plt.ylabel('Avergae velocity [km/s]', fontsize = 18)
plt.plot(meanages,u_av, '--', label = r'$\langle u \rangle$')
plt.plot(meanages,v_av, '--', label = r'$\langle v \rangle$')
plt.plot(meanages,w_av, '--', label = r'$\langle w \rangle$')
plt.legend()

plt.figure(2)
plt.rcParams['font.size'] = 20
fig = plt.figure(figsize=(20, 20))
fig.add_subplot(221)
plt.title('Velocity dispersions vs colour')
plt.xlabel(r'bp-rp', fontsize = 18)
plt.ylabel('Velocity dispersion [km/s]', fontsize = 18)
plt.plot(colorinterval, disp_x, '--', label = r'$\sigma^2_{\rm{u}}$')
plt.plot(colorinterval, disp_y, '--', label = r'$\sigma^2_{\rm{v}}$')
plt.plot(colorinterval, disp_z, '--', label = r'$\sigma^2_{\rm{w}}$')
plt.legend()

fig.add_subplot(222)
plt.title('Velocity dispersion vs mean age')
plt.xscale('log')
plt.xlabel(r'Mean age', fontsize = 18)
plt.ylabel('Velocity dispersion [km/s]', fontsize = 18)
plt.plot(meanages, disp_x, '--', label = r'$\sigma^2_{\rm{u}}$')
plt.plot(meanages, disp_y, '--', label = r'$\sigma^2_{\rm{v}}$')
plt.plot(meanages, disp_z, '--', label = r'$\sigma^2_{\rm{w}}$')
plt.legend()




In [ ]:
'''Calculating ratios in comparison with epicycle theory'''

ratios = []
for i in range(len(N)):
    rat = disp[i][1]/disp[i][0]
    ratios.append(rat)


In [ ]:
'''Calculation with new dispersions'''
Ratios = []

for i in range(len(N)):
    Ratios.append(disp_y[i]/disp_x[i])

plt.plot(colorinterval,Ratios)

In [ ]:
from scipy.optimize import curve_fit

disp_u_sqrt = []
disp_v_sqrt = []
disp_w_sqrt = []
disp_x_sqrt = []
disp_y_sqrt = []
disp_z_sqrt = []


for i in range(len(N)):
    disp_u_sqrt.append(np.sqrt(disp_u[i]))
    disp_v_sqrt.append(np.sqrt(disp_v[i]))
    disp_w_sqrt.append(np.sqrt(disp_w[i]))
    disp_x_sqrt.append(np.sqrt(disp_x[i]))
    disp_y_sqrt.append(np.sqrt(disp_y[i]))
    disp_z_sqrt.append(np.sqrt(disp_z[i]))
    
a1 = np.polyfit(np.log10(meanages),np.log10(disp_x_sqrt),1)
a2 = np.polyfit(np.log10(meanages),np.log10(disp_y_sqrt),1)
a3 = np.polyfit(np.log10(meanages),np.log10(disp_z_sqrt),1)

plt.rcParams['font.size'] = 20
fig = plt.figure(figsize=(10, 10))
#plt.xscale('linear')
#plt.yscale('linear')
#plt.plot(meanages,a1list)
#plt.plot(meanages,a2list)
#plt.plot(meanages,a3list)

#sigmalist = []
#for i in range(len(N)):
#    sigma = np.sqrt(disp_x[i] + disp_y[i] + disp_z[i])
#    sigmalist.append(sigma)
plt.ylabel(r'$\sigma$ [km/s]')
plt.xlabel('Mean age')
plt.xscale('log')
plt.yscale('log')
plt.plot(meanages,disp_x_sqrt, 'o', label = r'$\sigma_u$, $\alpha$ = {}'.format(a1[0]))
plt.plot(meanages,disp_y_sqrt, 'o', label = r'$\sigma_v$, $\alpha$ = {}'.format(a2[0]))
plt.plot(meanages,disp_z_sqrt, 'o', label = r'$\sigma_w$, $\alpha$ = {}'.format(a3[0]))
plt.legend()


a1,a2,a3


#plt.plot(meanages,disp_v_sqrt, 'o', label = r'$\sigma_{\rm{v}}$')
#plt.plot(meanages,disp_w_sqrt, 'o', label = r'$\sigma_{\rm{w}}$')

#plt.legend()

In [ ]:



plt.rcParams['font.size'] = 20
fig = plt.figure(figsize=(10, 10))
plt.xlabel(r'$\sigma^2_{\rm{u}}$')
plt.ylabel('Average velocities [km/s]')
plt.plot( disp_x,u_av, 'o', label = r'$\langle \vec{u} \rangle$')
plt.plot(disp_x,v_av, 'o')
plt.plot(disp_x,w_av, 'o')
plt.legend()

In [ ]:
plt.rcParams['font.size'] = 20
fig = plt.figure(figsize=(10, 10))
plt.xlabel(r'$bp-rp$')
plt.ylabel(r'Vertex deviation $\theta$ [rad]')
plt.plot(colorinterval, verticess, label = r'$\theta$')
plt.legend()

In [ ]:
'''
G_max = []
Gapp_max = []
distlist = []
for j in range(len(N)):    
    M_Gl = []
    for i in range(len(app)):
        M_G = app[i] + 5*np.log10(p[i]/100)
        M_Gl.append(M_G)
    max_Mg = np.max(app[j])
    
    G_max.append(max_Mg)
    Gapp_max.append(max_app)


    #max_app = np.max(app[j])
    #distance = 10**((max_app - max_Mg + 5)/5)
    #distlist.append(distance)

print(G_max)
print(Gapp_max)
print(distlist)
'''
x = [1,2,3,4,5,6,7,8,9,10,11]